In [1]:
!pip install datasets transformers evaluate

In [3]:
from datasets import load_dataset, Dataset
from typing import List, Tuple
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

MODEL_NAME = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')
print(train_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 701
})


In [4]:
MINIBATCH_SIZE = 32
tokenized_train_set = train_set.map(lambda it: tokenizer(it['text'], truncation=True, padding='max_length', max_length=128), batched=True)
tokenized_validation_set = validation_set.map(lambda it: tokenizer(it['text'], truncation=True, padding='max_length', max_length=128), batched=True)
print(tokenized_train_set)

Dataset({
    features: ['index_id', 'category', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 701
})


In [5]:
cls_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(predictions=predictions, references=labels, average='macro')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

list_of_categories = sorted(set(train_set['category']) | set(validation_set['category']) | set(test_set['category']))
indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

labeled_train_set = tokenized_train_set.add_column('label', [label2id[val] for val in tokenized_train_set['category']])
labeled_validation_set = tokenized_validation_set.add_column('label', [label2id[val] for val in tokenized_validation_set['category']])
print(labeled_train_set)

Dataset({
    features: ['index_id', 'category', 'text', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 701
})


In [6]:
!pip install protobuf==4.25.5 wandb==0.18.6

In [7]:
classifier = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=n_categories, id2label=id2label, label2id=label2id).cuda()

for param in classifier.parameters():
    param.data = param.data.contiguous()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=5e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=25,  # увеличил
    weight_decay=1e-2,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True #добавил
)

C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
pseudo_labeling_model_name = "cointegrated/rut5-base"
pseudo_labeling_model = AutoModelForSequenceClassification.from_pretrained(pseudo_labeling_model_name).cuda()
pseudo_labeling_tokenizer = AutoTokenizer.from_pretrained(pseudo_labeling_model_name)

def apply_pseudo_labeling(unlabeled_dataset, threshold=0.8):
    classification_pipeline = pipeline(
        'text-classification',
        model=pseudo_labeling_model,
        tokenizer=pseudo_labeling_tokenizer,
        device=0
    )

    texts = [f"текст: {text}" for text in unlabeled_dataset['text']]
    predictions = classification_pipeline(texts)

    pseudo_labels = []
    for prediction in predictions:
        if prediction['score'] >= threshold:
            pseudo_labels.append(label2id.get(prediction['label'], -1))
        else:
            pseudo_labels.append(-1)

    pseudo_labeled_dataset = unlabeled_dataset.add_column('label', pseudo_labels)
    pseudo_labeled_dataset = pseudo_labeled_dataset.filter(lambda example: example['label'] != -1)
    return pseudo_labeled_dataset

pseudo_labeled_dataset = apply_pseudo_labeling(test_set)

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at cointegrated/rut5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece to

In [10]:
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\snytk\AppData\Local\Temp\ipykernel_17212\1253556369.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: snytkinaleksandr2 (snytkinaleksandr2-ss). Use `wandb login --relogin` to force relogin


ClearML Task: created new task id=a38fa820031147138eee31e1bafef7cd
2024-11-18 20:22:49,037 - clearml.Task - INFO - Storing jupyter notebook directly as code
2024-11-18 20:22:49,074 - clearml.Repository Detection - WARNING - Can't get url information for git repo in C:\Users\snytk\DL
2024-11-18 20:22:49,123 - clearml.Repository Detection - WARNING - Can't get branch information for git repo in C:\Users\snytk\DL
2024-11-18 20:22:49,153 - clearml.Repository Detection - WARNING - Can't get commit information for git repo in C:\Users\snytk\DL
2024-11-18 20:22:49,266 - clearml.Repository Detection - WARNING - Can't get diff information for git repo in C:\Users\snytk\DL
ClearML results page: https://app.clear.ml/projects/9e8c4b01a416417dbbc804834e1e6868/experiments/a38fa820031147138eee31e1bafef7cd/output/log
2024-11-18 20:22:52,378 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


C:\Users\snytk\miniconda3\envs\python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.765076,0.796836
2,No log,0.528188,0.818139
3,No log,0.558542,0.839518


2024-11-18 20:23:30,870 - clearml.storage - INFO - Starting upload: C:\Users\snytk\AppData\Local\Temp\model_package.al1irx4q.zip => https://files.clear.ml/HuggingFace Transformers/Trainer.a38fa820031147138eee31e1bafef7cd/models/checkpoint-22.zip
2024-11-18 20:23:40,907 - clearml.storage - INFO - Uploading: 2040.62MB to C:\Users\snytk\AppData\Local\Temp\model_package.al1irx4q.zip


...

██▏                          9% | 174.00/2040.62 MB [08:12<1:28:46,  2.85s/MB]: 

2024-11-18 20:31:54,341 - clearml.Task - INFO - Waiting for previous model to upload (2 pending, https://files.clear.ml/HuggingFace Transformers/Trainer.a38fa820031147138eee31e1bafef7cd/models/checkpoint-66.zip)


██▎                          9% | 179.39/2040.62 MB [08:33<1:45:37,  3.40s/MB]: 

2024-11-18 20:32:24,343 - clearml.Task - INFO - Waiting for previous model to upload (2 pending, https://files.clear.ml/HuggingFace Transformers/Trainer.a38fa820031147138eee31e1bafef7cd/models/checkpoint-66.zip)


██▎                          9% | 179.47/2040.62 MB [08:43<2:28:37,  4.79s/MB]: 

In [ ]:
classifiсation_pipeline = pipeline('text-classification', model=classifier, tokenizer=tokenizer, device=0)

y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(validation_set['text'])))
y_true = validation_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(test_set['text'])))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))